Importing

In [1]:
###import the YOLOv8 model from ultralytics source
from ultralytics import YOLO

###image reading package
import cv2

###import other required packages
import time
import glob as glob


Load the Model

In [2]:
###load the pretrained yolov8n i.e., nano model for scratch training on custom dataset
#tsd_model = YOLO("yolov8n.pt")

###load pretrained model on the current dataset from the last checkpoint and not the best checkpoint if want to train on more checkpoints or resume training
tsd_model = YOLO("G:\\Projects\\ADAS\\TSD\\Output\\Model_Outputs\\weights\\last.pt")

Training

In [3]:
###start the training from pretrained model or scratch
###refer doc : https://docs.ultralytics.com/modes/train/
###refer : https://github.com/ultralytics/ultralytics/issues/802#issuecomment-1627557388
'''results = tsd_model.train(
   data="G:\\Projects\\ADAS\\TSD\\Inputs\\Normal_Augmented_Images\\GTSD_Data.yaml",
   imgsz=640,
   epochs=110,
   batch=4,
   name="G:\\Projects\\ADAS\\TSD\\Output\\Model_Outputs",
   device=0,
   val=True,
   plots=True
)'''

results = tsd_model.train(
   data="G:\\Projects\\ADAS\\TSD\\Inputs\\Normal_Augmented_Images\\GTSD_Data.yaml",
   imgsz=640,
   epochs=30,
   batch=4,
   name="G:\\Projects\\ADAS\\TSD\\Output\\Model_Outputs_Continue",
   device=0,
   val=True,
   plots=True
)

###resume the training if previous training got stopped in between
###refer : https://github.com/ultralytics/ultralytics/issues/802#issuecomment-1616631122
'''results = tsd_model.train(
   resume=True,
   data="G:\\Projects\\ADAS\\TSD\\Inputs\\Normal_Augmented_Images\\GTSD_Data.yaml",
   imgsz=640,
   epochs=30,   ###more number of epochs needed
   batch=4,
   name="G:\\Projects\\ADAS\\TSD\\Output\\Model_Outputs",
   device=0,
   val=True,
   plots=True
)'''

New https://pypi.org/project/ultralytics/8.1.9 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.184  Python-3.8.0 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=G:\Projects\ADAS\TSD\Output\Model_Outputs\weights\last.pt, data=G:\Projects\ADAS\TSD\Inputs\Normal_Augmented_Images\GTSD_Data.yaml, epochs=30, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=G:\Projects\ADAS\TSD\Output\Model_Outputs_Continue, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=T

'results = tsd_model.train(\n   resume=True,\n   data="G:\\Projects\\ADAS\\TSD\\Inputs\\Normal_Augmented_Images\\GTSD_Data.yaml",\n   imgsz=640,\n   epochs=30,   ###more number of epochs needed\n   batch=4,\n   name="G:\\Projects\\ADAS\\TSD\\Output\\Model_Outputs",\n   device=0,\n   val=True,\n   plots=True\n)'

Testing

In [2]:
all_time = time.time()
###load the trained model
#tsd_model = YOLO("G:\\Projects\\ADAS\\TSD\\Output\\Model_Outputs\\weights\\best.pt")
tsd_model = YOLO("G:\\Projects\\ADAS\\TSD\\Output\\Model_Outputs_Continue\\weights\\best.pt")

###directory where all the images and its annotation json are present
DIR_TEST = "G:\\Projects\\ADAS\\TSD\\Inputs\\Normal_Augmented_Images\\test\\images"
test_images = glob.glob(f"{DIR_TEST}/*.jpg")
print(f"Test instances : {len(test_images)}")

###all 46 classes
CLASSES = ['ANIMALS', 'CONSTRUCTION', 'CYCLES CROSSING', 'DANGER', 'NO ENTRY', 'PEDESTRIAN CROSSING', 'SCHOOL CROSSING', 'SNOW', 'STOP', 'bend left', 'bend right', 'bend', 'give way', 'go left or straight', 'go left', 'go right or straight', 'go right', 'go straight', 'keep left', 'keep right', 'no overtaking (trucks)', 'no overtaking -trucks-', 'no overtaking', 'no traffic both ways', 'no trucks', 'priority at next intersection', 'priority road', 'restriction ends (overtaking (trucks))', 'restriction ends (overtaking)', 'restriction ends -overtaking -trucks--', 'restriction ends -overtaking-', 'restriction ends 80', 'restriction ends', 'road narrows', 'roundabout', 'slippery road', 'speed limit 100', 'speed limit 120', 'speed limit 20', 'speed limit 30', 'speed limit 50', 'speed limit 60', 'speed limit 70', 'speed limit 80', 'traffic signal', 'uneven road']

###define the detection threshold, any detection having score below this will be discarded
detection_threshold = 0.40

###loop through all test images
for i in range(len(test_images)):
    each_img_time = time.time()
    ###get the image file name for saving output later on
    image_name = test_images[i].split('\\')[-1]
    print("Image Name = ", image_name)
    ###read the image using cv2
    image = cv2.imread(test_images[i])
    ###get image size for further groundtruth box resizing
    hght, wdth, *_ = image.shape
    ###take a copy of image to draw boxes and classes name with score
    orig_image = image.copy()

    ###use the already load model for prediction
    ###more info : https://docs.ultralytics.com/modes/predict/
    outputs = tsd_model(image)

    ###loop through the results/outputs
    for op in outputs:
        
        ###check if the boxes/predictions are available by looping
        for box in range(len(op.boxes.xyxy)):
            
            ###check if the conf equals or exceeds the threshold value
            if(op.boxes.conf[box] >= detection_threshold):
                
                ###draw model predictions on image
                cv2.rectangle(orig_image,
                            (int(op.boxes.xyxy[box][0]), int(op.boxes.xyxy[box][1])),
                            (int(op.boxes.xyxy[box][2]), int(op.boxes.xyxy[box][3])),
                            (0, 255, 0,), 2)
                cv2.putText(orig_image, CLASSES[int(op.boxes.cls[box])], 
                            (int(op.boxes.xyxy[box][0]), int(op.boxes.xyxy[box][1]-5)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 
                            2, lineType=cv2.LINE_AA)
                cv2.putText(orig_image, str(round(float(op.boxes.conf[box]),2)), 
                            (int(op.boxes.xyxy[box][0]), int(op.boxes.xyxy[box][1]-15)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 
                            2, lineType=cv2.LINE_AA)
        
    ###save the predicted images
    path = "G:\\Projects\\ADAS\\TSD\\Output\\Test_Predictions\\" + image_name
    cv2.imwrite(path, orig_image)

    print(f"Image {i+1} Done with Time : ", str(time.time() - each_img_time))
    print('-'*50)

print('\nTEST PREDICTIONS COMPLETE WITH TIME : ',str(time.time() - all_time))

Test instances : 54
Image Name =  00026_jpg.rf.4c55e8a72c668d5a0c4a948580757cd9.jpg



0: 640x640 1 speed limit 30, 4.0ms
Speed: 17.1ms preprocess, 4.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 CONSTRUCTION, 1 go straight, 8.0ms
Speed: 4.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 give way, 1 roundabout, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 keep right, 5.0ms
Speed: 3.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 priority road, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 keep right, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 slippery road, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 keep left, 4.0ms
Speed: 2.0ms preprocess, 4.0ms in

Image 1 Done with Time :  2.5054619312286377
--------------------------------------------------
Image Name =  00028_jpg.rf.975675b221390ef694e07dfa6eca3bea.jpg
Image 2 Done with Time :  0.02292323112487793
--------------------------------------------------
Image Name =  00032_jpg.rf.36c53d3b779076659978df5719da5b39.jpg
Image 3 Done with Time :  0.015409708023071289
--------------------------------------------------
Image Name =  00041_jpg.rf.b6ba02dfdcd60a59334e5f24efaea448.jpg
Image 4 Done with Time :  0.014990568161010742
--------------------------------------------------
Image Name =  00062_jpg.rf.939bfe50885300a78222574527e78860.jpg
Image 5 Done with Time :  0.014947652816772461
--------------------------------------------------
Image Name =  00066_jpg.rf.e5fa3f32335ddd495114f305e3895c7d.jpg
Image 6 Done with Time :  0.014913797378540039
--------------------------------------------------
Image Name =  00070_jpg.rf.4fee81d10981620631421d78e620d2e1.jpg
Image 7 Done with Time :  0.015

0: 640x640 2 speed limit 80s, 4.0ms
Speed: 3.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 no overtaking (trucks), 1 traffic signal, 5.0ms
Speed: 3.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 speed limit 20, 4.0ms
Speed: 3.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 keep left, 3 keep rights, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 traffic signal, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 speed limit 30, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 speed limit 70, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 traffic signal, 4.0ms
Spee

Image 13 Done with Time :  0.018936634063720703
--------------------------------------------------
Image Name =  00129_jpg.rf.d417cc6b9a74b4763f5351a803842a82.jpg
Image 14 Done with Time :  0.01694321632385254
--------------------------------------------------
Image Name =  00138_jpg.rf.01010e01ef667e6517a62db5e67027f5.jpg
Image 15 Done with Time :  0.012956619262695312
--------------------------------------------------
Image Name =  00159_jpg.rf.e3fc812f4b9dba2efd9763c024c61a63.jpg
Image 16 Done with Time :  0.014950037002563477
--------------------------------------------------
Image Name =  00164_jpg.rf.ef65f8ed316efa2750568ec6ad102898.jpg
Image 17 Done with Time :  0.012956619262695312
--------------------------------------------------
Image Name =  00166_jpg.rf.abf740a0af265f78d41491ae42a30d0e.jpg
Image 18 Done with Time :  0.013953208923339844
--------------------------------------------------
Image Name =  00181_jpg.rf.7e8a9233c1f10e491ef367c955210194.jpg
Image 19 Done with Time

...

Image 26 Done with Time :  0.013953685760498047
--------------------------------------------------
Image Name =  00268_jpg.rf.397d9b0adea2709b80799f9a1ad1622b.jpg
Image 27 Done with Time :  0.012956619262695312
--------------------------------------------------
Image Name =  00271_jpg.rf.1f0d5cd2570141f8c8069d42bd01db1a.jpg
Image 28 Done with Time :  0.013953208923339844
--------------------------------------------------
Image Name =  00273_jpg.rf.8ce15e75e635916a58822522f27bae39.jpg
Image 29 Done with Time :  0.013953447341918945
--------------------------------------------------
Image Name =  00281_jpg.rf.6cffa9b862d95dbe6985b6cc8d7cd89a.jpg
Image 30 Done with Time :  0.013953208923339844
--------------------------------------------------
Image Name =  00294_jpg.rf.93d866e7dc76ee9af9fd323f2afa62ca.jpg
Image 31 Done with Time :  0.01195979118347168
--------------------------------------------------
Image Name =  00300_jpg.rf.80a3756a3acf20ac54925653d8bd79da.jpg
Image 32 Done with Time

0: 640x640 1 roundabout, 1 speed limit 120, 4.0ms
Speed: 3.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 speed limit 100s, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 speed limit 100, 2 speed limit 120s, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 speed limit 80, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 keep rights, 4.0ms
Speed: 3.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 speed limit 100s, 1 speed limit 80, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 speed lim

Image 40 Done with Time :  0.012956619262695312
--------------------------------------------------
Image Name =  00396_jpg.rf.f84d5e20613cfea1e9be2e80f6386e6f.jpg
Image 41 Done with Time :  0.014949798583984375
--------------------------------------------------
Image Name =  00415_jpg.rf.fd53b8b8394359890a049d98e0fe73fc.jpg
Image 42 Done with Time :  0.011960506439208984
--------------------------------------------------
Image Name =  00428_jpg.rf.90f07d70585076e3e541458d59693e68.jpg
Image 43 Done with Time :  0.014949798583984375
--------------------------------------------------
Image Name =  00439_jpg.rf.899fe386bccaca2e045525cfd03bb936.jpg
Image 44 Done with Time :  0.01331019401550293
--------------------------------------------------
Image Name =  00448_jpg.rf.5def67d7c0eaaf0e89f72a54f86b09ad.jpg
Image 45 Done with Time :  0.013954639434814453
--------------------------------------------------
Image Name =  00483_jpg.rf.13c2d0fdd6bc43845c0e177e53e1ebe6.jpg
Image 46 Done with Time

Speed: 2.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Image 54 Done with Time :  0.012956857681274414
--------------------------------------------------

TEST PREDICTIONS COMPLETE WITH TIME :  3.29209566116333
